# Reservoir Computing (RC) for PDE — Colab Notebook
このノートブックは本レポジトリの実装をGoogle Colabで実行できるように整理したものです。
Mackey-Glass時系列予測とMNIST分類タスクを順に実行します。

In [ ]:
# (任意) 必要に応じて依存関係をインストールしてください。
# Colabには多くのパッケージが事前インストールされています。
# !pip install -q torch torchvision matplotlib


In [ ]:
import os
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import torch
from torchesn import utils
from torchesn.nn import ESN
from torchvision import datasets, transforms

REPO_ROOT = os.path.abspath(os.getcwd())
if REPO_ROOT not in sys.path:
    sys.path.insert(0, REPO_ROOT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Repo root:', REPO_ROOT)
print('Device:', device)


## Mackey-Glass 時系列予測

In [ ]:
torch.set_default_dtype(torch.double)
dtype = torch.double

data_path = os.path.join(REPO_ROOT, 'datasets', 'mg17.csv')
if dtype == torch.double:
    data = np.loadtxt(data_path, delimiter=',', dtype=np.float64)
else:
    data = np.loadtxt(data_path, delimiter=',', dtype=np.float32)

X_data = np.expand_dims(data[:, [0]], axis=1)
Y_data = np.expand_dims(data[:, [1]], axis=1)
X_data = torch.from_numpy(X_data).to(device)
Y_data = torch.from_numpy(Y_data).to(device)

trX = X_data[:5000]
trY = Y_data[:5000]
tsX = X_data[5000:]
tsY = Y_data[5000:]

washout = [500]
input_size = output_size = 1
hidden_size = 500
loss_fcn = torch.nn.MSELoss()


In [ ]:
start = time.time()

trY_flat = utils.prepare_target(trY.clone(), [trX.size(0)], washout)

model = ESN(input_size, hidden_size, output_size)
model.to(device)

model(trX, washout, None, trY_flat)
model.fit()
output, hidden = model(trX, washout)
print('Training error:', loss_fcn(output, trY[washout[0]:]).item())

output, hidden = model(tsX, [0], hidden)
print('Test error:', loss_fcn(output, tsY).item())
print('Ended in', time.time() - start, 'seconds.')


In [ ]:
predictions = output.detach().cpu().squeeze().numpy()
targets = tsY.detach().cpu().squeeze().numpy()

plt.figure(figsize=(10, 4))
plt.plot(targets, label='Target')
plt.plot(predictions, label='Prediction')
plt.title('Mackey-Glass Time Series Prediction')
plt.xlabel('Time Step')
plt.ylabel('Value')
plt.legend()
plt.tight_layout()
plt.show()


## MNIST 分類タスク

In [ ]:
torch.set_default_dtype(torch.float)

def Accuracy_Correct(y_pred, y_true):
    labels = torch.argmax(y_pred, 1).type(y_pred.type())
    correct = len((labels == y_true).nonzero())
    return correct


def one_hot(y, output_dim):
    onehot = torch.zeros(y.size(0), output_dim, device=y.device)

    for i in range(output_dim):
        onehot[y == i, i] = 1

    return onehot


def reshape_batch(batch):
    batch = batch.view(batch.size(0), batch.size(1), -1)
    return batch.transpose(0, 1).transpose(0, 2)


loss_fcn = Accuracy_Correct

batch_size = 256
input_size = 1
hidden_size = 500
output_size = 10
washout_rate = 0.2


In [ ]:
data_path = os.path.join(REPO_ROOT, 'datasets')
train_iter = torch.utils.data.DataLoader(
    datasets.MNIST(data_path, train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))])),
    batch_size=batch_size, shuffle=True, num_workers=1, pin_memory=True)

test_iter = torch.utils.data.DataLoader(
    datasets.MNIST(data_path, train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))])),
    batch_size=batch_size, shuffle=False, num_workers=1, pin_memory=True)


In [ ]:
start = time.time()

model = ESN(input_size, hidden_size, output_size,
            output_steps='mean', readout_training='cholesky')
model.to(device)

for batch in train_iter:
    x, y = batch
    x = x.to(device)
    y = y.to(device)

    x = reshape_batch(x)
    target = one_hot(y, output_size)
    washout_list = [int(washout_rate * x.size(0))] * x.size(1)

    model(x, washout_list, None, target)
    model.fit()

tot_correct = 0
tot_obs = 0

for batch in train_iter:
    x, y = batch
    x = x.to(device)
    y = y.to(device)

    x = reshape_batch(x)
    washout_list = [int(washout_rate * x.size(0))] * x.size(1)

    output, hidden = model(x, washout_list)
    tot_obs += x.size(1)
    tot_correct += loss_fcn(output[-1], y.type(torch.get_default_dtype()))

print('Training accuracy:', tot_correct / tot_obs)


In [ ]:
viz_images = None
viz_labels = None
viz_preds = None

for batch in test_iter:
    x, y = batch
    x = x.to(device)
    y = y.to(device)

    x_images = x.detach().cpu()
    x_seq = reshape_batch(x)
    washout_list = [int(washout_rate * x_seq.size(0))] * x_seq.size(1)

    output, hidden = model(x_seq, washout_list)
    tot_obs += x_seq.size(1)
    tot_correct += loss_fcn(output[-1], y.type(torch.get_default_dtype()))
    if viz_images is None:
        viz_images = x_images
        viz_labels = y.detach().cpu()
        viz_preds = torch.argmax(output[-1].detach().cpu(), dim=1)

print('Test accuracy:', tot_correct / tot_obs)
print('Ended in', time.time() - start, 'seconds.')


In [ ]:
if viz_images is not None:
    num_images = min(16, viz_images.size(0))
    fig, axes = plt.subplots(4, 4, figsize=(6, 6))
    for idx in range(num_images):
        ax = axes[idx // 4, idx % 4]
        image = viz_images[idx].squeeze().numpy()
        ax.imshow(image, cmap='gray')
        ax.set_title(f'P:{viz_preds[idx].item()} T:{viz_labels[idx].item()}')
        ax.axis('off')
    plt.suptitle('MNIST Predictions (P: Predicted, T: True)')
    plt.tight_layout()
    plt.show()
